In [1]:
import cv2 as cv
import numpy as np
from glob import glob
from sklearn.utils import shuffle
import time
import pandas as pd
import pickle
from config import efficientNet_config
from models import efficientNetV2B0_model, efficientNetV2B3_model
from utils import plt_heatmap, plt_roccurve

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

In [2]:
model = efficientNetV2B0_model()
model.load_weights('../model/202211/20221102.weights')

2023-01-11 13:07:56.813667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-11 13:07:56.814136: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-11 13:07:56.819158: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-11 13:07:56.819570: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-11 13:07:56.819929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Function  (None, 1280)             5919312   
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 5,920,593
Trainable params: 5,859,985
Non-trainable params: 60,608
_________________________________________________________________


In [3]:
height  = efficientNet_config['height_B0']
width   = efficientNet_config['width_B0']

In [4]:
testdata = "test_h"
test_dir      = '../dataset_test_everyframes/{}/'.format(testdata)
## everyframes
img_paths = sorted(glob(test_dir + "*.png"))

start_time = time.time()

pred_results = []
img_names = []
for i,img_path in enumerate(img_paths):
    # imege
    img_name  = img_path.split("/")[-1]
    img_array = cv.imread(img_path)
    img_array = cv.resize(img_array,(height,width))     # resize to (224,224)
    
    
    pred_result = model.predict(np.array([img_array]))
    pred_result = np.where(pred_result > 0.5, 1, 0)
    
    if pred_result[0][0]!=0:
        print(pred_result[0][0])
        pred_results.append(pred_result[0][0])
        img_names.append(img_name)


end_time = time.time()

print(end_time-start_time)

2023-01-11 13:08:00.913260: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 21ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 23ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [=======================

In [5]:
stage1_everyframes_result = pd.DataFrame({"stage1_pred_result":pred_results,"img_name":img_names})

In [6]:
with open('../dataset_test_everyframes/{}_stage1_everyframes_result_20230110.pk'.format(testdata), 'wb') as f:
    pickle.dump(stage1_everyframes_result, f)